In [13]:
import open3d as o3d
import numpy as np
import copy
import random

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [20]:
source = o3d.io.read_point_cloud("ICP/r1.pcd")
target = o3d.io.read_point_cloud("ICP/r2.pcd")

# Used for downsampling.
voxel_size = 0.05

# Show models side by side
draw_registrations(source, target)

In [15]:
####
# Downsample and find features here
####
voxel_size = 0.05
radius_feature = voxel_size * 5
# Downsample
source_downsampled = source.voxel_down_sample(voxel_size=voxel_size)
target_downsampled = target.voxel_down_sample(voxel_size=voxel_size)

print(np.asarray(source_downsampled.points).shape)
print(np.asarray(target_downsampled.points).shape)

# Estimating normals of the pointcloud points using
print("Recompute the normal of the downsampled point cloud")
source_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
target_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# o3d.visualization.draw_geometries([source, target])

# Compute features
source_fpfh = o3d.registration.compute_fpfh_feature(source_downsampled, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
target_fpfh = o3d.registration.compute_fpfh_feature(target_downsampled, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))

print(source_fpfh.data.shape)
print(source_fpfh.data[1, 1])

(4760, 3)
(3440, 3)
Recompute the normal of the downsampled point cloud
(33, 4760)
0.0


In [81]:
search_kd = o3d.geometry.KDTreeFlann(target_fpfh)

def random_points_kabsch(source_downsampled, target_downsampled, source_fpfh, target_fpfh, search_kd):

    source_down = np.asarray(source_downsampled.points)
    target_down = np.asarray(target_downsampled.points)

    # Get 3 random points
    number_of_nearest_neighbours = 10

    source_points = []
    target_points = []
    for i in range(5):
        random_point = random.randint(0, source_fpfh.data.shape[0]-1)
        source_point_fpfh = source_fpfh.data[:, random_point]
        source_points.append(np.asarray(source_downsampled.points)[random_point, :])
        [_, idx, _] = search_kd.search_knn_vector_xd(source_point_fpfh, number_of_nearest_neighbours)
        random_match = random.randint(0, number_of_nearest_neighbours-1)
        target_points.append(np.asarray(target_downsampled.points)[idx[random_match], :])

    source_points = np.array(source_points)
    target_points = np.array(target_points)

    # Calculate centroids
    centroid_source = np.sum(source_down, axis=0)/source_down.shape[0]
    centroid_target = np.sum(target_down, axis=0)/target_down.shape[0]

    # Subtract centroids from random chosen points
    source_points_cen = source_points - centroid_source
    target_points_cen = target_points - centroid_source

    # Cross variance
    C = np.transpose(source_points_cen) @ target_points_cen

    # SVD on covariance
    U, S, V = np.linalg.svd(C)

    R = U @ np.transpose(V)
    T = centroid_target - R @ centroid_source

    return R, T, source_points, target_points

R, T, source_points_cen, target_points_cen = random_points_kabsch(source_downsampled, target_downsampled, source_fpfh, target_fpfh, search_kd)
print(f'Rotation is: {R}, \n\n---------\n\nTranslation is: {T}')

Rotation is: [[-0.61443129 -0.76591287 -0.18934538]
 [-0.62953042  0.62059486 -0.46749702]
 [ 0.47556875 -0.16804612 -0.86347835]], 

---------

Translation is: [5.16161616 2.77595231 2.20057141]


In [82]:
run_ransac = 10

distance_ = np.inf
for i in range(run_ransac):
    R, T, source_points, target_points = random_points_kabsch(source_downsampled, target_downsampled, source_fpfh, target_fpfh, search_kd)

    source_points = np.transpose(R @ np.transpose(source_points)) + T
    sum_square_diff = np.sum((target_points - source_points)**2)
    

    if distance_ > sum_square_diff:
        print(sum_square_diff)
        distance_ = sum_square_diff
        transform_matrix = np.zeros([4, 4])
        transform_matrix[:3, :3] = R
        transform_matrix[:3, 3] = T
        transform_matrix[3, 3] = 1
        
        #source_downsampled = source_downsampled.transform(transform_matrix)


draw_registrations(source, target, transform_matrix, True)



11.898521645400946
9.50421409551969
7.038849798616178
5.024971065255992
